In [9]:
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
with open('../data/pets.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file)

In [11]:
info = soup.find_all('article', class_='tabber__panel')
tier_data = info[:-1]
token_data = info[-1]

In [ ]:
def extract_pet_data(tier_data):
    data = []
    for tier in tier_data:
        tier_name = tier.find('h3').get_text(strip=True)
        table = tier.find('table', class_='wikitable')
        if not table:
            continue

        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if not cells:
                continue

            if cells[0].has_attr('rowspan'):
                rowspan = int(cells[0]['rowspan'])
                name = cells[0].find('a')['title']
                attack = cells[1].get_text(strip=True)
                health = cells[2].get_text(strip=True)
                pack_ref = cells[-1].find_all('a')
                packs = [ref.get('title', '').strip() for ref in pack_ref]

                ability_rows = [row] + [row.find_next_sibling('tr', recursive=False) for _ in range(rowspan - 1)]
                for idx, ability_row in enumerate(ability_rows):
                    ability_cells = ability_row.find_all('td')
                    if idx == 0:
                        ability_level = ability_cells[3].text.strip()
                        ability_desc = ability_cells[4].text.strip()
                    else:
                        ability_level = ability_cells[0].text.strip()
                        ability_desc = ability_cells[1].text.strip()

                    data.append({
                        'Name': name,
                        'Attack': attack,
                        'Health': health,
                        'Ability Level': ability_level,
                        'Ability Description': ability_desc,
                        'Packs': packs,
                        'Tier': tier_name
                    })
    pets_df = pd.DataFrame(data)
    return pets_df

def extract_token_data(token_data):
    data = []
    table = token_data.find('table', class_='wikitable')
    if not table:
        return pd.DataFrame()

    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if not cells:
            continue

        name = cells[0].find('a')['title']

        if cells[1].has_attr('colspan'):
            

        print(cells)

In [13]:
extract_token_data(token_data)

[<td><span style="white-space: nowrap;"><a href="/wiki/Bee" title="Bee"><img alt="" data-file-height="212" data-file-width="230" decoding="async" height="23" loading="lazy" src="/images/thumb/1/10/Bee_Icon.png/25px-Bee_Icon.png?d9bf94" width="25"/></a><span style="visibility:hidden; color:transparent;"> </span><span style="white-space: normal;"><a href="/wiki/Bee" title="Bee">Bee</a></span></span>
</td>, <td colspan="3">1/1
</td>, <td>N/A
</td>, <td>Any Pet with <span style="white-space: nowrap;"><a href="/wiki/Honey" title="Honey"><img alt="" data-file-height="230" data-file-width="209" decoding="async" height="25" loading="lazy" src="/images/thumb/f/f3/Honey_Icon.png/23px-Honey_Icon.png?b8d5b6" width="23"/></a><span style="visibility:hidden; color:transparent;"> </span><span style="white-space: normal;"><a href="/wiki/Honey" title="Honey">Honey Perk</a></span></span> once it faints.
</td>, <td>
</td>]
[<td><span style="white-space: nowrap;"><a href="/wiki/Loyal_Chinchilla" title="Loy